# tornado-简介

Tornado 是一个Python web框架和异步网络库，起初由 FriendFeed(后被facebook收购，目前tornado由facebook开发维护)开发。由于非阻塞的特性，它在处理Http长连接、websocket等保持连接时间较长的请求时，并发能力很强。

Tornado 大体上可以被分为4个主要的部分:
* web框架 (包括创建web应用的 RequestHandler 类，还有很多其他支持的类).
* HTTP的客户端和服务端实现 (HTTPServer and AsyncHTTPClient).
* 异步网络库 (IOLoop and IOStream), 为HTTP组件提供构建模块，也可以用来实现其他协议.
* 协程库 (tornado.gen) 允许异步代码写的像同步代码一样直观，而不用链式回调的方式.

tornado依靠底层基于epoll(Linux)或者kqueue(BSD和MAC OSX)的IOLoop实现非阻塞IO，而且经过FriendFeed的实践，已经证明是绝对优秀可靠的非阻塞IO框架，再加上协程特性，让基于他的异步代码可以像阻塞多线程框架的同步代码一样易读易维护。

### RequestHandler的生命周期

tornado.web.RequestHandler的生命周期是initialize() -> prepare() -> get()/post() -> on_finish()

以下方法可以都可以被重载,一般根据实际需要来决定是否重载:

* initialize() 在构造函数后调用，一般用于定义参数，**不可异步**

* prepare() 在具体的get()/post()/put()/delete()等执行前调用，一般用于加载登录信息，过滤请求等，**可异步**

* get()/post()/put()/delete() 处理请求

* on_finish() 请求后的清理，保存缓存，session等,该方法不能传递任何数据到客户端，所以**不能操作cookie，可异步**

* get_current_user() 第一次使用实例中的currentuser参数且为None时会调用该方法，**不能异步**，如果需要通过异步来获取的，请写在prepare()中.

### tornado的一些限制

1. 单线程server
tornado不仅仅是一个web framework，还是一个简易的web server，这让它可以直接作为一个server来接收处理http请求，而不需要依靠wsgi容器。但是这个webserver过于简单，只支持单进程，所以在**生产环境中，官方推荐的多进程多主机部署**，启动多个tornado server实例分别监听不同端口，在上层通过类似nginx的成熟高效的http server来做负载均衡，将请求转发到合适端口的tornado实例中。（参考tornado官方文档运行部署篇）

2. session
tornado本身没有实现session，因为它是解决C10K这类高并发问题，由于cpython的全局锁限制，作为一个单线程的server无法利用多核的特性，所以官方推荐多进程多实例甚至多主机部署以此来充分利用多核心来处理高并发，一般跨进程的session同步势必用到第三方工具，所以tornado实现单实例的session没有太大的意义。